In [9]:
import numpy as np
import pandas as pd
from glob import glob

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import binom
from sklearn.metrics import confusion_matrix

import mle
import flmp
from libfunc import to_table_body

import warnings
warnings.filterwarnings('ignore')

# Load Data

In [10]:
# get paths to data files
file_paths = glob("./data/*.txt")

# load all data into a single array
data = np.array([np.loadtxt(fname) for fname in file_paths])
N, M, K = data.shape

# define number of samples for each subject
n_samples = 24 

# number of samples for simulated data
N_SAMPLES = 100

# Model Distributions on Data with FLMP and MLE
Fit MLE and FLMP to the data, create the corresponding distribution and sample from these to create simulated data.

In [11]:
# compute parameters and generate samples
samples = {
    "FLMP": np.zeros((N, M, K)),
    "MLE": np.zeros((N, M, K))
}
params = []
for i, d in enumerate(data):
    # get parameters for MLE and FLMP fits
    _, c_A, c_V, std_A, std_V = mle.fit(np.ones(4), d, n_samples)
    _, tA, tV = flmp.fit(np.ones(K*2), d, n_samples, K)

    # save parameters
    params.append(np.concatenate([tA, tV, np.array([c_A, std_A, c_V, std_V])]))

    # sample from FLMP
    p_A, p_V, p_AV = flmp.compute_probs(tA, tV)
    samples["FLMP"][i] = binom.rvs(N_SAMPLES, np.vstack([p_A.T, p_V.T, p_AV]))
    
    # sample from MLE
    p_A, p_V, p_AV = mle.compute_probs(c_A, c_V, np.exp(std_A), np.exp(std_V))
    samples["MLE"][i]  = binom.rvs(N_SAMPLES, np.vstack([p_A.T, p_V.T, p_AV]))

# Fit FLMP and MLE on Simulated Data

In [12]:
tmp = {
    "Fitting Model": [],
    "Sampling Model": [], 
    "Subject": [],
    "NLL": []
}
for model in samples:
    for i, Subject in enumerate(samples[model]):
        # compute negative log-likelihood for MLE fit on both simulated data
        nll, _, _, _, _ = mle.fit(np.ones(4), Subject, N_SAMPLES)
        tmp["Fitting Model"].append("MLE")
        tmp["Sampling Model"].append(model)
        tmp["Subject"].append(f"{i+1}")
        tmp["NLL"].append(nll)
        
        # compute negative log-likelihood for FLMP fit on both simulated data
        nll, _, _ = flmp.fit(np.ones(K*2), Subject, N_SAMPLES, K)
        tmp["Fitting Model"].append("FLMP")
        tmp["Sampling Model"].append(model)
        tmp["Subject"].append(f"{i+1}")
        tmp["NLL"].append(nll)

In [13]:
df_fit = pd.DataFrame.from_dict(tmp)
df_fit.groupby("Subject").min()

,Fitting Model,Sampling Model,NLL
Subject,,,
1,FLMP,FLMP,68.253611
2,FLMP,FLMP,46.293495
3,FLMP,FLMP,69.025277
4,FLMP,FLMP,69.094061
5,FLMP,FLMP,56.119569


In [42]:
df_fit

,Fitting Model,Sampling Model,Subject,NLL
0,MLE,FLMP,1,224.070378
1,FLMP,FLMP,1,68.253611
2,MLE,FLMP,2,174.899092
3,FLMP,FLMP,2,46.293495
4,MLE,FLMP,3,157.037318
5,FLMP,FLMP,3,69.025277
6,MLE,FLMP,4,177.417985
7,FLMP,FLMP,4,69.094061
8,MLE,FLMP,5,152.409924
9,FLMP,FLMP,5,56.119569


In [43]:
for subject in df_fit['Subject'].unique():
    subject_data = df_fit[df_fit['Subject'] == subject]
    for sm in subject_data['Sampling Model'].unique():
        smd = subject_data[subject_data['Sampling Model'] == sm]
        print(smd[smd['NLL'] == smd['NLL'].min()])

  Fitting Model Sampling Model Subject        NLL
1          FLMP           FLMP       1  68.253611
   Fitting Model Sampling Model Subject       NLL
10           MLE            MLE       1  78.77272
  Fitting Model Sampling Model Subject        NLL
3          FLMP           FLMP       2  46.293495
   Fitting Model Sampling Model Subject        NLL
12           MLE            MLE       2  63.441832
  Fitting Model Sampling Model Subject        NLL
5          FLMP           FLMP       3  69.025277
   Fitting Model Sampling Model Subject        NLL
14           MLE            MLE       3  79.137655
  Fitting Model Sampling Model Subject        NLL
7          FLMP           FLMP       4  69.094061
   Fitting Model Sampling Model Subject       NLL
16           MLE            MLE       4  82.96229
  Fitting Model Sampling Model Subject        NLL
9          FLMP           FLMP       5  56.119569
   Fitting Model Sampling Model Subject        NLL
18           MLE            MLE       5  58.5

In [49]:
plot_options = {
    'annot': True,
    'xticklabels': ['FLMP', 'MLE'],
    'yticklabels': ['FLMP', 'MLE'],
}
y_true = ["FLMP"]*5 + ["MLE"]*5
y_pred = ["FLMP"]*5 + ["MLE"]*5

# get confusion matrix and normalized confusion matrix
cm = confusion_matrix(y_true, y_pred)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# display confusion matrices
plt.figure(figsize=(6, 6))
sns.set(font_scale=1) 
sns.heatmap(cm, **plot_options)

plt.tick_params(axis='both', which='major', labelsize=13)  # Adjust to fit

plt.xlabel('Best Fitting Model', fontsize=13)
plt.ylabel('Sampling Model', fontsize=13)
plt.title('Confusion Matrix', fontsize=15)
plt.tight_layout()
plt.savefig("./visualizations/confusion_matrix.png", bbox_inches="tight")

# Print out stuff for Latex report

In [16]:
# print out parameters in latex format
print('Parameters for all subjects')
print(to_table_body(params))

# display data for a single subject
SUBJECT_NUMBER = 0
print(f"\nSimulated data for 'Subject {SUBJECT_NUMBER+1}'\n(ignore first column)")
print(to_table_body(np.hstack([
    samples['FLMP'][SUBJECT_NUMBER], 
    samples['MLE'][SUBJECT_NUMBER]
]), num_float=False).replace('Subject ', ''))

# extract dataframe for the single subject of negative log-likelihoods
print(f'\nNegative log-likelihoods for "Subject {SUBJECT_NUMBER+1}"')
df_fit_ = df_fit.reset_index()
df_fit_ = df_fit_[df_fit_['Subject'].astype(int) == SUBJECT_NUMBER + 1].set_index(["Fitting Model"])
df_fit_[df_fit_.columns[df_fit_.columns.isin(['Sampling Model', 'NLL'])]]

Parameters for all subjects
Subject 1 & -3.31 & -2.29 & -1.52 & 2.46 & 3.21 & -2.97 & 0.84 & 2.91 & 3.58 & 3.83 & 3.31 & -0.10 & 1.86 & -0.13\\
Subject 2 & -3.68 & -1.83 & -0.32 & 2.58 & 4.83 & -5.29 & 0.91 & 3.20 & 5.09 & 5.87 & 3.02 & -0.27 & 1.98 & -0.53\\
Subject 3 & -2.65 & -1.74 & -0.69 & 2.62 & 5.41 & -2.00 & 0.25 & 1.54 & 2.37 & 2.62 & 2.90 & -0.10 & 2.05 & 0.13\\
Subject 4 & -4.12 & -3.11 & -2.04 & 2.25 & 4.62 & -2.52 & -1.19 & -0.31 & 1.26 & 2.12 & 3.36 & -0.36 & 3.11 & 0.13\\
Subject 5 & -5.43 & -3.93 & -1.42 & 2.92 & 6.24 & -3.01 & 0.42 & 1.73 & 2.42 & 3.05 & 3.32 & -0.53 & 2.07 & -0.08

Simulated data for 'Subject 1'
(ignore first column)
1 & 6 & 12 & 14 & 88 & 94 & 0 & 7 & 26 & 67 & 99\\
2 & 3 & 70 & 99 & 95 & 98 & 7 & 58 & 90 & 100 & 100\\
3 & 1 & 0 & 1 & 36 & 62 & 1 & 6 & 12 & 41 & 66\\
4 & 8 & 19 & 27 & 94 & 99 & 4 & 17 & 45 & 71 & 93\\
5 & 45 & 61 & 86 & 100 & 100 & 21 & 41 & 83 & 89 & 99\\
6 & 55 & 80 & 91 & 100 & 100 & 52 & 76 & 95 & 94 & 100\\
7 & 58 & 87 & 91 & 10

,Sampling Model,NLL
Fitting Model,,
MLE,FLMP,224.070378
FLMP,FLMP,68.253611
MLE,MLE,78.772720
FLMP,MLE,93.058033
